In [ ]:
import warnings
warnings.filterwarnings('ignore')
from helpers import *
import numpy as np
import plotly.express as px

np.random.seed(42)

# Estimating Pandemic Risk

Here, we will estimate the expected number of deaths this century from the following types of pandemics:
1. Natural pandemics (based on estimates from [Marani et al. 2021](https://doi.org/10.1073/pnas.2105482118))
2. Accidental pandemics (based on estimates from [Klotz 2021](https://armscontrolcenter.org/wp-content/uploads/2017/04/LWC-paper-final-version-for-CACNP-website.pdf))
3. Deliberate pandemics (based on estimates from [Esvelt 2022](https://dam.gcsp.ch/files/doc/gcsp-geneva-paper-29-22?_gl=1*ieur8b*_ga*MTk1NzA0MTU3My4xNjk2NzcyODA0*_ga_Z66DSTVXTJ*MTY5Njc3MjgwNC4xLjAuMTY5Njc3MjgwNi41OC4wLjA.))

Thank you to the Cambridge Biosecurity Group, especially Joshua Blake, for fruitful discussions that informed lots of the estimates here. However, any errors are my own and the estimates I make here are very rough.

In [ ]:
# Define vars
params = Params()

We will will estimate the expected number of deaths this century i.e. from 2024 to 2100. For simplicity, we'll assume a fixed population of 9.2 billion people over this time period (based on [UN population projections](https://www.worldometers.info/world-population/world-population-projections/)):

$
\text{Average population} = \frac{\text{Starting population} + \text{Population in 2100}}{2} = \frac{8.0 \text{ billion} + 10.3 \text{ billion}}{2} = 9.2 \text{ billion}
$

For the estimates, we'll be using Monte Carlo simulations. These parameters which are used for multiple of the estimates are shown below:

In [ ]:
params.print_category('Global')

## 1. Estimating Natural Pandemic Risk

We'll be estimating the expected number of deaths from natural pandemics this century. This analysis will involve three main sections:

1. Load data
2. Load the GPD model
3. Use the GPD model to estimate the expected number of deaths from natural pandemics this century

To esimate natural pandemic risk, we'll use the following parameters from [Marani et al.](https://doi.org/10.1073/pnas.2105482118):

In [ ]:
params.print_category('Natural')

num_simulations = params.Global.num_simulations.val

### 1.1. Load Data

[Marani et al.](https://doi.org/10.1073/pnas.2105482118) assembled a dataset of epidemics that have occured since 1500.

For their analysis, which we'll replicate next, they only used data from 1600 due to reliability. They also excluded epidemics that were ongoing at the time of writing (eg: AIDS/HIV, malaria, and COVID-19) or were ended by pharmaceuticals (eg: smallpox) which excluded all epidemics after the end of WWII. This is because they likely have a different distribution.

The full dataset is visualised below with the time period from 1600 to 1944, that we'll use for subsequent analyses, highlighted in green.

In [ ]:
marani_xls = params.Natural.dataset.val
marani_df, disease_totals = load_and_preprocess_natural_data(marani_xls)
color_map = generate_color_map(disease_totals)
fig = plot_disease_timeline(marani_df, disease_totals, color_map)

### 1.2. Load the GPD model

Epidemic intensity refers to the number of deaths due to an epidemic in a given year. When we plot this intensity against its exceedance probability, it provides insights into the likelihood of an epidemic of a given intensity occuring in a given year. The exceedance probability is the probability that the epidemic intensity will be exceeded in a given year.

Marani et al. demonstrated that this relationship between epidemic intensity and exceedance probability is well-described by the Generalized Pareto Distribution (GPD), a family of continuous probability distributions often used to model the tail of the distribution of extreme events.

The GPD is defined mathematically defined as:

$
H(i) = P_0 + (1 - P_0) \times \left(1 + \frac{\xi \times (i - \mu)}{\sigma}\right)^{-\frac{1}{\xi}}
$

Where:
- $ H(i) $ is the exceedance probability of the epidemic intensity $ i $.
- $ \mu $ is the threshold parameter.
- $ \sigma $ is the scale parameter.
- $ \xi $ is the shape parameter.
- $ P_0 $ is the probability for intensities below the threshold $ \mu $.

The epidemic intensity is plotted against its exceedance probability below, including the GPD using parameter estimates from [Marani et al.](https://doi.org/10.1073/pnas.2105482118). Note: both axes are on a log scale.

<!-- #### Understanding Exceedance Probability

The exceedance probability of the yearly maximum epidemic intensity, $ H_1(i) $, is given by $ H_1(i) = 1 - P_1(i) $. This value represents the likelihood that an extreme novel epidemic (irrespective of its causative disease) with an intensity equal to or greater than $ i $, will occur anywhere in the world within a year.

To put this into perspective, consider an epidemic with an intensity similar to the 1918-1920 "Spanish flu." For this epidemic, with an intensity of $ i = 5.7 $ deaths per thousand per year, the yearly probability of exceedance is represented by $ H_1(i = 5.7 \text{‰/year}) $. -->

<!-- Epidemic intensity refers to the number of deaths due to an epidemic in a given year. When we plot this intensity against its exceedance probability, it provides insights into the likelihood of extreme epidemic events. Specifically, the exceedance probability indicates the probability that the epidemic intensity will be exceeded in a given year.

Marani et al. demonstrated that this relationship between epidemic intensity and exceedance probability is well-described by the Generalized Pareto Distribution (GPD). The GPD is particularly useful for modeling extreme events, making it a suitable choice for representing extreme epidemic intensities.

The Generalized Pareto Distribution (GPD) is mathematically defined as:



Marani et al. showed that the when the epidemic intesity, that is the number of deaths per year, is plotted against the exceedance probability, which is the probability that the epidemic intensity will be exceeded in a given year, the data is well described by a generalized Pareto distribution (GPD). The GPD is a probability distribution that is often used to model extreme events. The GPD is defined as:

$
\text{GPD}(x) = \begin{cases}
\frac{1}{\sigma} \left(1 + \frac{\xi x}{\sigma} \right)^{-\frac{1}{\xi} - 1} & \text{if } \xi \neq 0 \\
\frac{1}{\sigma} e^{-\frac{x}{\sigma}} & \text{if } \xi = 0
\end{cases}
$

where $\sigma$ is the scale parameter and $\xi$ is the shape parameter. The exceedance probability is defined as:

$
\text{Exceedance probability} = \frac{\text{Number of epidemics with intensity greater than } x}{\text{Total number of epidemics}}
$

The GPD is visualised below: -->

In [ ]:
mu = params.Natural.mu.val
sigma = params.Natural.sigma.val
xi = params.Natural.xi.val

marani_1e_5_df = marani_df[(marani_df["Intensity (deaths per thousand/year)"] > 1e-5) & (marani_df["Start Year"].between(1600, 1944))]
# marani_1e_5_df = marani_df[(marani_df["Intensity (deaths per thousand/year)"] > 1e-5) & (marani_df["Start Year"] > 1944)]

fig = plot_exceedance_probability(
        marani_df=marani_1e_5_df,
        plot_gpd=True,
        plot_lognorm=False,
        mu=mu,
        sigma=sigma,
        xi=xi
    )

### 1.3 Use the GPD Model to Estimate the Expected Number of Deaths from Natural Pandemics this Century

To estimate the yearly expected intensity, I first truncated the GPD at 17.8 deaths per thousand per year, as this is the highest credible reports of deaths for a pandemic, 100 million deaths from 1918 influenza divided by population at the time, divided by the three-year duration, similar to the method used by [Glennerster et al.](https://www.nber.org/system/files/working_papers/w30565/w30565.pdf). I then calculated the intergral of the GPD from 0 to this value.

In [ ]:
population = params.Global.population.val
max_intensity = params.Natural.max_intensity.val
vaccines_multiplier = params.Natural.vaccines_multiplier.val

intensities= np.linspace(mu, max_intensity, 1000)
# gpd_values = genpareto.sf(intensities - mu, xi, scale=sigma)
gpd_values = compute_gpd(intensities, mu, sigma, xi)
expected_yearly_intensity = np.trapz(intensities * gpd_values, intensities) * vaccines_multiplier
E_natural_deaths_annual = expected_yearly_intensity * (population / 1e3)

fig = plot_exceedance_probability(intensities, gpd_values, log_axis=False, title_text=f"Expected yearly intensity = {E_natural_deaths_annual/1e6:.1f} million deaths")

To get the expected number of natural pandemic deaths this century, I simply multiplied the yearly expected intensity by the number of years remaining this century:

$
E[\text{#deliberate_deaths}] = E[\text{#deliberate_deaths_per_year}] \times \text{#years}
$

In [ ]:
num_years = params.Global.num_years.val
E_natural_deaths = E_natural_deaths_annual * num_years

display_text(f"Expected number of deaths from natural epidemics this century = <strong>{E_natural_deaths/1e6:.1f} million deaths</strong>")

### Limitations

The results of this analysis are similar to those of [Madhav et al.](https://www.cgdev.org/sites/default/files/estimated-future-mortality-pathogens-epidemic-and-pandemic-potential.pdf) who reported an annual expected 2.5 million deaths or **197.6 million deaths** this century. However there are several limitations to this analysis. 

The limitations of this analysis include:
- The estimate may be inaccurate as:
    - The GPD may not be a good fit for the data. It is particular hard to determine the probability of extreme events from a small sample size. 
- The estimate may be an overestimate as: 
    - Based on visual inspection, the GPD appears to overestimate the probability of extreme events.
    - The GPD is fit to data from before the widespread use of antibiotics and vaccines. Although the effect of vaccines has been accounted for approximately, using estimates for COVID-19, the effects of therapeutics and other medical advances have not been accounted for.
- The estimate may be an underestimate as:
    - It does not account for factors that may increase natural pandemic risk, such as: increasing population density and mobility, climate change, deforestation, and antibiotic resistance.

## 2. Estimating Accidental Pandemic Risk

We'll be examining the potential risk of a pandemic from the accidental release of viruses from research facilities. This analysis will involve three main sections:

1. Probability a single facility in a single year seeds a pandemic
2. Expected number of accidental pandemics this century
3. Expected number of deaths from accidental pandemics this century

In [ ]:
params.print_category('Accidental')

### 2.1 Probability a single facility in a single year seeds a pandemic

The probability that a single facility seeds a pandemic in a single year is given by:

$
P_{\text{single_pandemic}} = P_{\text{release}} \times P_{\text{seeds_pandemic}}
$

Where:
- $P_{\text{release}}$ is the probability of community release from a single facility in a single year.
- $P_{\text{seeds_pandemic}}$ is the probability that a virus release seeds a pandemic. Since $P_{\text{seeds_pandemic}}$ is given as a range, we will sample from a uniform distribution between 0.05 and 0.4.

In [ ]:
P_seeds_pandemic_min, P_seeds_pandemic_max = params.Accidental.P_seeds_pandemic.val
P_release = params.Accidental.P_release.val
accidental_colour = params.Accidental.colour.val

P_seeds_pandemic = np.random.uniform(P_seeds_pandemic_min, P_seeds_pandemic_max, num_simulations)
P_single_pandemic = P_release * P_seeds_pandemic

# Plot the distribution of P_single_pandemic
fig = plot_P_single_pandemic_hist(P_single_pandemic, accidental_colour)

### 2.2. Expected number of accidental pandemics this century

The expected number of pandemics seeded by any facility in a $num\_years$ is given by:

$
E[\text{#Pandemics}] = P_{\text{single_pandemic}} \times \text{#years} \times \text{#facilities}
$

Where:
- $\text{#years}$ is the number of years in the simulation (typically set to 100 for a century).
- $\text{#facilities}$ is the number of facilities in the world.
- $P_{\text{single_pandemic}}$ is the probability that a single facility seeds a pandemic in a single year.

In [ ]:
num_facilities = params.Accidental.num_facilities.val
growth_rate = params.Accidental.growth_rate.val

# Store the number of facilities over time
facilities_over_time = []

for year in range(num_years):
    adjusted_facilities = int(num_facilities * (1 + growth_rate) ** year)
    facilities_over_time.append(adjusted_facilities)

# Convert to a numpy array
facilities_over_time = np.array(facilities_over_time)

def plot_facilities_over_time(facilities_over_time, accidental_colour, start_year=2023):
    years = list(range(start_year, start_year + len(facilities_over_time)))
    fig = go.Figure(data=[
        go.Scatter(x=years, y=facilities_over_time, mode='lines', name='Number of Facilities', line=dict(color=accidental_colour))
    ])
    fig.update_layout(title="Number of facilities over time", xaxis_title="Years", yaxis_title="Number of Facilities")
    fig.show()
    return fig

# Example usage
fig = plot_facilities_over_time(facilities_over_time, accidental_colour)


In [ ]:
E_accidental_pandemics = []

for year in range(num_years):
    E_accidental_pandemics_yearly = P_single_pandemic * facilities_over_time[year]
    E_accidental_pandemics.append(E_accidental_pandemics_yearly)

# Convert to a numpy array
E_accidental_pandemics = np.array(E_accidental_pandemics)

# Plot the distribution of E_accidental_pandemics
E_accidental_pandemics = np.sum(E_accidental_pandemics, axis=0)
fig = plot_E_accidental_pandemics_hist(E_accidental_pandemics, accidental_colour)

### 2.3. Expected number of deaths from accidental pandemics this century

For each pandemic, the number of deaths is calculated as:

$
E[\text{#Deaths} ]= E[\text{#Pandemics}] \times \text{Population} \times \text{Infection Rate} \times \text{CFR}
$

Where:
- $\text{Population}$ is the world population at the time of the pandemic.
- $\text{Infection Rate}$ is the infection rate of the pandemic.
- $\text{CFR}$ is the case fatality rate of the pandemic.

In [ ]:
infection_rate = params.Accidental.infection_rate.val
fatality_rate = params.Accidental.fatality_rate.val

E_accidental_deaths = E_accidental_pandemics * population * infection_rate * fatality_rate

# Plot the distribution of E_accidental_deaths
fig = plot_E_accidental_deaths_hist(E_accidental_deaths, accidental_colour)

### Limitations

Limitations of this analysis include:
- The estimate may be inaccurate as:
    - The results are highly dependent on the probabilities of release computed in [Klotz 2020](https://armscontrolcenter.org/wp-content/uploads/2020/03/Quantifying-the-risk-9-17-Supplementary-material-at-end.pdf)
- The results may be an overestimate as:
    - We are assuming the number of facilities will continue to grow exponentially at the same rate they've grown since 1970
- The results may be an underestimate as:
    - We are only considering the risk from highly pathogenic avian influenza (HPAI) viruses, and not other types of viruses
    - We are only considering the risk from research facilities, and not other sources of accidental release

## 3. Estimating Deliberate Pandemic Risk

To estimate the expected number of deaths from deliberate pandemics this century, we'll perform the following steps:
1. Estimate the number of bioterrorists this century<br>
    1.1. Using previous terrorist attacks (Method 1)<br>
    1.2. Using historical examples of bioterrorists (Method 2)<br>
2. Estimating the number of bioterrorists this century (accounting for individuals retraining)
3. Estimating the number of deaths (when pandemic blueprints become available)

In [ ]:
params.print_category('Deliberate')

In [ ]:
gtd_xls = params.Deliberate.dataset.val
deaths_per_attack = params.Deliberate.deaths_per_attack.val
population_us_1995 = params.Deliberate.population_us_1995.val
num_indv_capability = params.Deliberate.num_indv_capability.val
num_indv_capability_intent_last_century = params.Deliberate.num_indv_capability_intent_last_century.val
retrain_indv_multiplier_min, retrain_indv_multiplier_max = params.Deliberate.retrain_indv_multiplier.val
deliberate_multiplier_min, deliberate_multiplier_max = params.Deliberate.deliberate_multiplier.val
num_years_blueprints_min, num_years_blueprints_max = params.Deliberate.num_years_until_blueprints.val
deliberate_colour = params.Deliberate.colour.val

### 3.1 Estimating the number of bioterrorists this century
We'll use two different methods to estimate the number of individuals who already have the capability and intent i.e. scientists turned bioterrorists.

#### 3.1.1 Using previous terrorist attacks (Method 1)

$
E[\text{#Bioterrorists}] = \text{Individuals With Capability} \times \text{Fraction of People With Intent}
$

For the number of individuals with capability, we'll use the number of new individuals who learn to assemble a virus each year from [Esvelt 2022](https://dam.gcsp.ch/files/doc/gcsp-geneva-paper-29-22).

For the fraction of these people with intent, we'll estimate this based on previous terrorist incidents in the US. Given the widespread access to firearms in the US we'll assume that any individual who wants to cause mass harm, defined here as killing at least two people, can do so. Specifically, we'll look at the number of terrorist events in the Global Terrorism Database ([GTD](https://www.start.umd.edu/gtd/)) in the US between 1970 and 2020.

In [ ]:
# Load and preprocess data
gtd_df = load_and_preprocess_deliberate_data(gtd_xls, deaths_per_attack)
num_events = len(gtd_df)

# Plot 
fig = plot_deaths_per_attack_scatter(gtd_df, deaths_per_attack, num_events)

In [ ]:
frac_invd_intent = num_events / population_us_1995
display_text(format_intent_fraction(frac_invd_intent))

E_num_bioterrorists_per_year_1 = num_indv_capability * frac_invd_intent

#### 3.1.2 Using historical examples of bioterrorists (Method 2)
For this method we'll look at the historical base rate of bioterrorists. Historically terrorists have failed to use bioweapons. In the last century, arguably the individuals who came the closest to releasing an infectious agent were members from the terrorist organisations Aum Shinrikyo and al-Qaeda. Here, we'll assume that they failed due a lack of capability and they would have succeded using biotechnology from this century.

In [ ]:
E_num_bioterrorists_per_year_2 = num_indv_capability_intent_last_century / 100

In [ ]:
E_num_bioterrorists_per_year = np.mean([E_num_bioterrorists_per_year_1, E_num_bioterrorists_per_year_2])
display_text(f"""
    Expected number of bioterrorists this century
    = (Method 1 + Method 2) / 2
    = ({E_num_bioterrorists_per_year_1 * num_years:.1f} + {E_num_bioterrorists_per_year_2 * num_years:.1f}) / 2
    = <strong> {E_num_bioterrorists_per_year * num_years:.1f} scientists turned bioterrorists </strong>
    """
    )

### 3.2 Estimating the number of bioterrorists this century (accounting for individuals retraining)
Here, we'll adjust our estimate to account for terrorists who become bioterrorists using estimate from [Esvelt 2023](https://dam.gcsp.ch/files/doc/securing-civilisation-against-catastrophic-pandemics-gp-31).

In [ ]:
# Account for retraining
retrain_indv_multiplier = np.random.uniform(retrain_indv_multiplier_min, retrain_indv_multiplier_max, num_simulations)
E_num_bioterrorists_per_year = E_num_bioterrorists_per_year * retrain_indv_multiplier
E_num_bioterrorists_this_century = E_num_bioterrorists_per_year * num_years

display_text(f"Expected Number of Bioterrorists this century (accounting for individuals retraining) = <strong> {np.mean(E_num_bioterrorists_this_century):.1f} total bioterrorists </strong>") 

### 3.3 Estimating the number of deaths (when pandemic blueprints become available)

Here we assume that:
1. Blueprints for a pathogen with pandemic potential will become available within the next 50 years
2. Blueprints for a pathogen with pandemic potential have an equal probability of becoming available at any point in this 50-year window
3. Bioterrorists are unable to cause a deliberate pandemic until such blueprints become available
4. That a deliberate pandemic would be 1-10x worse than that caused by a pathogen with 1918 IFR due to factors such as multiple pathogens and/or releases

In [ ]:
year_blueprints_available = np.random.uniform(num_years_blueprints_min, num_years_blueprints_max, num_simulations)
deliberate_multiplier = np.random.uniform(deliberate_multiplier_min, deliberate_multiplier_max, num_simulations)
E_deliberate_deaths = np.zeros((num_simulations, num_years))
for i in range(num_simulations):
    for j in range(num_years):
        if j >= year_blueprints_available[i]:
            E_deliberate_deaths[i, j] = E_num_bioterrorists_per_year[i] * population * infection_rate * fatality_rate * deliberate_multiplier[i]

E_deliberate_deaths_avg = (np.mean(E_deliberate_deaths, axis=0))

# total_individuals = calculate_individual_capability(params)
# fig = plot_capability_growth(params, deliberate_colour)

def plot_E_deliberate_deaths_over_time(E_deliberate_deaths_avg, num_years, deliberate_colour, start_year=2023):
    fig = go.Figure(data=[
        go.Scatter(x=list(range(start_year, start_year + num_years)), y=E_deliberate_deaths_avg, mode='lines', name='Expected Deaths', line=dict(color=deliberate_colour))
    ])
    fig.update_layout(title="Expected number of deaths from deliberate pandemics over this century", xaxis_title="Years", yaxis_title="Expected Deaths")
    fig.show()
    return fig

fig = plot_E_deliberate_deaths_over_time(E_deliberate_deaths_avg, num_years, deliberate_colour)

In [ ]:
fig = plot_E_deliberate_deaths_hist(np.sum(E_deliberate_deaths, axis=1), deliberate_colour)

# Estimating intervention cost-effectiveness

Here we will compare the cost-effectiveness of different interventions. The cost-effectiveness of an example global health and animal welfare intervention are shown below.

In [ ]:
params.print_category('Interventions')
average_dalys_per_life_saved = params.Interventions.average_dalys_per_life_saved.val

## 1. DNA Synthesis Screening

To evaluate the effectiveness of DNA synthesis screening we'll estimate the overall probability of preventing a bioterrorist attack using DNA synthesis screening, which can be decomposed into several contributing factors. Each factor represents a critical step or condition necessary for the screening to be effective. We'll assume that all DNA synthesis providers do effective screening, either voluntarily or due to regulation.


In [ ]:
params.print_category('DNA_Screening')

P_no_benchtop = params.DNA_Screening.P_no_benchtop.val
P_no_academic_approval = params.DNA_Screening.P_no_academic_approval.val
P_pathogen_in_database = params.DNA_Screening.P_pathogen_in_database.val
P_screening_effective = params.DNA_Screening.P_screening_effective.val
dna_screening_cost = params.DNA_Screening.dna_screening_cost.val

The equation for this probability is as follows:

$
P(\text{preventing attack with screening}) = P(\text{no benchtop}) \times P(\text{no academic approval}) \times P(\text{pathogen in database}) \times P(\text{screening effective})
$

<!-- Where:

- $ P(\text{provider screens}) $ is the probability that the DNA synthesis provider conducts the screening.
- $ P(\text{no academic approval}) $ is the probability that the order does not have academic approval (assuming orders with academic approval bypass the sc$ening).
- $ P(\text{pathogen in database}) $ is the probability that the pathogen is listed in the screening database.
- $ P(\text{screening effective}) $ is the probability that the screening, if conducted, successfully identifies and stops a bioterrorist attack.

Gi$n the following estimated probabilities for each factor:

- $ P(\text{provider screens}) = 0.8 $
- $ P(\text{no academic approval}) = 0.75 $
- $ P(\text{pathogen in database}) = 0.5 $
- $ P(\text{screening effective}) = 0.92 $

The overall probability of preventing an attack with screening is calculated as:

$ P(\text{preventing attack with screening}) = 0.8 \times 0.75 \times 0.5 \times 0.92 $ -->

In [ ]:
P_screening_effective = P_no_benchtop * P_no_academic_approval * P_pathogen_in_database * P_screening_effective

display_text(f"DNA synthesis screening = <strong> {P_screening_effective * 100:.1f}% effective </strong>")

We'll assume that screening prevents some percentage of deliberate attacks as defined above. However, we'll also assume that the attacks that are not prevented are the most severe, as the most sophisticated, e.g. better resourced, bioterrorists will be able to evade screening.

In [ ]:
sophistication_threshold = deliberate_multiplier_max * P_screening_effective
E_deliberate_deaths_with_screening = np.zeros(E_deliberate_deaths.shape)

for i in range(num_simulations):    
    for j in range(num_years):
        if j >= year_blueprints_available[i]:
            attack_probability = E_num_bioterrorists_per_year[i] * (deliberate_multiplier[i] > sophistication_threshold)
            E_deliberate_deaths_with_screening[i, j] = attack_probability * population * infection_rate * fatality_rate * deliberate_multiplier[i]

In [ ]:
# Sum the expected deaths over all years
E_deliberate_deaths_this_century = np.sum(E_deliberate_deaths, axis=1)
E_deliberate_deaths_with_screening_this_century = np.sum(E_deliberate_deaths_with_screening, axis=1)

# Plot the expected deaths with and without screening
plot_comparative_E_deliberate_deaths_hist(E_deliberate_deaths_this_century, E_deliberate_deaths_with_screening_this_century, deliberate_colour)

In [ ]:
lives_saved_with_screening = int(np.mean(E_deliberate_deaths_this_century - E_deliberate_deaths_with_screening_this_century))
dalys_saved_with_screening = lives_saved_with_screening * average_dalys_per_life_saved
cost_in_thousands = dna_screening_cost / 1000
cost_effectiveness = dalys_saved_with_screening / cost_in_thousands

display_text(f"Cost-effectiveness of DNA synthesis screening = <strong> {cost_effectiveness:,.0f} DALYs / $1000 </strong>")

## 2. Metagenomic Sequencing

To evaluate the cost-effectiveness of metagenomic sequencing we'll use estimates for the [ThreatNet](https://pubmed.ncbi.nlm.nih.gov/37367195/) system for early oubtreak detection which relies on metagenomic sequencing of clinical samples.

In [ ]:
params.print_category('Sequencing')

P_containment = params.Sequencing.P_containment.val
threatnet_cost_min, threatnet_cost_max = params.Sequencing.threatnet_cost.val
us_pop_frac_global = params.Sequencing.us_pop_frac_global.val

It is difficult to quantify the benefits of early outbreak detection as it is not a direct intervention but rather a tool that can be used to inform interventions. However, we'll assume that the main benefit of early outbreak detection is that it allows interventions such as contact tracing to be implemented earlier with the potential to contain the outbreak before it becomes a pandemic. For this, we'll rely on modelling estimates from [Hellewell et al. 2020](https://www.thelancet.com/journals/langlo/article/PIIS2214-109X(20)30074-7).

In [ ]:
E_pandemic_deaths = E_natural_deaths + np.mean(E_accidental_deaths) + np.mean(E_deliberate_deaths_this_century)

lives_saved_with_sequencing = E_pandemic_deaths * P_containment
dalys_saved_with_sequencing = lives_saved_with_sequencing * average_dalys_per_life_saved + population * 0.3 * 1/52 * num_years

display_text(f"Expected number of deaths from pandemics this century = <strong> {E_pandemic_deaths:,.0f} </strong>")

Here, we estimate the cost of global sequencing by considering the ThreatNet cost and adjusting for the global population and the cost of running the system until the end of the century.

In [ ]:
threatnet_cost = np.random.uniform(threatnet_cost_min, threatnet_cost_max, num_simulations)
sequencing_cost = threatnet_cost * 1 / us_pop_frac_global * num_years

sequencing_cost_in_thousands = sequencing_cost / 1000

display_text(f"Cost of global sequencing = <strong> ${np.mean(sequencing_cost):,.0f} </strong>")

Finally, we evaluate the cost-effectiveness of metagenomic sequencing in terms of DALYs saved per $1000 spent. It's possible that the cost of metagenomic sequencing will decrease in the coming decades (e.g. ~10x), see [Sequencing Roadmap](http://sequencing-roadmap.org/). Also, the primary benefit of metagenomic sequencing may be preventing subtle biothreats ([Gopal et al. 2023](https://dam.gcsp.ch/files/doc/securing-civilisation-against-catastrophic-pandemics-gp-31)) and longter-term benefits which are not accounted for here

In [ ]:
cost_effectiveness = dalys_saved_with_sequencing / sequencing_cost_in_thousands

display_text(f"Cost-effectiveness of metagenomic sequencing = <strong> {np.mean(cost_effectiveness):,.0f} DALY / $1000 </strong>")

## 3. Broad-spectrum vaccines
Here we'll estimate the cost-effectiveness of pan-coronavirus and pan-influenza vaccines.

We'll assume that a 10x increase in current funding for both pan-coronavirus and pan-influenza vaccines will accelerate the development of both broad-spectrum vaccine by 5 years. I define current funding here as the single largest investment in broad-spectrum vaccines I could find, which is CEPI's $200M investment in pan-coronavirus vaccines.

In [ ]:
params.print_category('Broad_Vaccines')

additional_vaccine_years = params.Broad_Vaccines.additional_vaccine_years.val
covid_excess_deaths = params.Broad_Vaccines.covid_excess_deaths.val
time_to_update_vaccine_min, time_to_update_vaccine_max = params.Broad_Vaccines.time_to_update_vaccine.val
current_flu_vaccine_effectiveness_min, current_flu_vaccine_effectiveness_max = params.Broad_Vaccines.current_flu_vaccine_effectiveness.val
broad_vaccine_effectiveness_min, broad_vaccine_effectiveness_max = params.Broad_Vaccines.broad_vaccine_effectiveness.val
annual_global_flu_deaths_min, annual_global_flu_deaths_max = params.Broad_Vaccines.annual_global_flu_deaths.val
broad_cov_research_funding = params.Broad_Vaccines.broad_cov_research_funding.val
dalys_lost_per_seasonal_flu_death = params.Broad_Vaccines.dalys_lost_per_seasonal_flu_death.val

We'll quantify the benefits for:
1. **Coronavirus and influenza pandemics:** estimating the lives saved due to the immediate availability of vaccines, reducing the impact at the onset of pandemics.
2. **Seasonal influenza epidemics:** calculating the additional lives saved by having a vaccine that is more effective than current seasonal influenza vaccines, as it is antigneically matched to the circulating strain in all flu seasons.

In [ ]:
# Extract flu-related data from Marani dataset
flu_data = marani_1e_5_df[marani_1e_5_df["Disease"] == "Influenza"]
total_flu_deaths = flu_data["# deaths (raw)"].sum().astype(int)

# Combine COVID-19 and influenza deaths
combined_flu_covid_deaths = covid_excess_deaths + total_flu_deaths

# Calculate the fraction of deaths due to flu and COVID-19
fraction_flu_covid_deaths = combined_flu_covid_deaths / (E_natural_deaths + covid_excess_deaths)

# Estimate the expected number of deaths from natural coronavirus and influenza epidemics
expected_flu_covid_deaths = fraction_flu_covid_deaths * E_natural_deaths_annual

# Average duration of epidemics from the dataset
average_epidemic_duration = flu_data["Duration"].mean()

In [ ]:
# Initialize arrays to store the results for each year
lives_saved_seasonal = np.zeros((additional_vaccine_years, num_simulations))
lives_saved_pandemic = np.zeros((additional_vaccine_years, num_simulations))

for year in range(additional_vaccine_years):
    # Randomly sample vaccine effectiveness and time to update vaccine for this year
    current_flu_vaccine_effectiveness = np.random.uniform(current_flu_vaccine_effectiveness_min, current_flu_vaccine_effectiveness_max, num_simulations)
    broad_vaccine_effectiveness = np.random.uniform(broad_vaccine_effectiveness_min, broad_vaccine_effectiveness_max, num_simulations)
    time_to_update_vaccine = np.random.uniform(time_to_update_vaccine_min, time_to_update_vaccine_max, num_simulations)

    # Estimate annual global flu deaths for this year
    annual_global_flu_deaths = estimate_deaths_vectorized(current_flu_vaccine_effectiveness, annual_global_flu_deaths_min, annual_global_flu_deaths_max)

    # Compute lives saved by broad-spectrum vaccine availability and improved seasonal vaccine effectiveness
    time_saved_fraction = time_to_update_vaccine / average_epidemic_duration
    lives_saved_immediate_vaccine = expected_flu_covid_deaths * time_saved_fraction * (1 - vaccines_multiplier)
    additional_lives_saved = annual_global_flu_deaths * ((broad_vaccine_effectiveness - current_flu_vaccine_effectiveness) / current_flu_vaccine_effectiveness)

    # Store the lives saved for seasonal and pandemic vaccines separately
    lives_saved_seasonal[year] = additional_lives_saved
    lives_saved_pandemic[year] = lives_saved_immediate_vaccine

# Aggregate the lives saved over all years
total_lives_saved_seasonal = np.sum(lives_saved_seasonal, axis=0)
total_lives_saved_pandemic = np.sum(lives_saved_pandemic, axis=0)

# Plot the results
fig = plot_lives_saved_comparison(total_lives_saved_seasonal, total_lives_saved_pandemic)

To estimate the development cost for broad-spectrum vaccines, we'll consider CEPI's $200M investment in a pan-coronavirus vaccine as a reference (this is largest investment in broad vaccines that I could find). We'll assume that a 20x increase in this investment covers the development of both pan-coronavirus and pan-influenza vaccines.

In [ ]:
# Estimated cost for developing and implementing broad-spectrum vaccines
estimated_vaccine_cost = broad_cov_research_funding * 2 * 10  # 10x funding increase assumption
estimated_vaccine_cost_thousands = estimated_vaccine_cost / 1000

In [ ]:
# Convert total lives saved to DALYs
dalys = total_lives_saved_seasonal * dalys_lost_per_seasonal_flu_death + total_lives_saved_pandemic * average_dalys_per_life_saved

# Cost-Effectiveness Calculation
cost_effectiveness = np.mean(dalys / estimated_vaccine_cost_thousands)

# Display the cost-effectiveness result
display_text(f"Cost-effectiveness of broad-spectrum vaccines = <strong> {cost_effectiveness:,.0f} DALY / $1000 </strong>")